In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt 
import lasio
import os
from tqdm import tqdm

pd.set_option("display.precision", 3)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_rows', 4)
pd.set_option('display.max_columns', 15)

# wellheads

In [3]:
folder_path = r"C:\Petrel\well_traj"  # replace with your folder path
file_names = os.listdir(folder_path)

df_list = []
for well in file_names:
    file_path = os.path.join(folder_path, well)
    las = lasio.read(file_path)
    df = las.df()
    df['well'] = well.split('.')[0]
    df_list.append(df)
df_survey = pd.concat(df_list).reset_index()

In [4]:
wellhead = df_survey.groupby('well')[['well','X','Y']].apply(lambda x: x.iloc[0]).reset_index(drop=True)

In [ ]:
wellhead[wellhead.X.isna()]

In [6]:
wellhead_v2 = wellhead[~wellhead.well.isin(['B35','B43'])]

In [7]:
add1 = df_survey[df_survey.well == 'B43'][['well','X','Y']].iloc[1:2]
add2 = df_survey[df_survey.well == 'B35'][['well','X','Y']].iloc[1:2]
wellhead_v2 = pd.concat([wellhead_v2, add1, add2]).reset_index(drop=True).drop_duplicates()

In [12]:
# wellhead_v2.to_csv('wellhead.prn', sep='\t', index=False)

# gr_cube

In [14]:
folder_path = r'C:\Petrel\gr_cube'  # replace with your folder path
file_names = os.listdir(folder_path)

df_list = []
for well in file_names:
    file_path = os.path.join(folder_path, well)
    las = lasio.read(file_path)
    df = las.df()
    df['well'] = well.split('.')[0]
    df_list.append(df)
df_gr_cube = pd.concat(df_list).reset_index()

In [ ]:
df_gr_cube = df_gr_cube.rename(columns={'DEPT': 'MD', 'GR_CUBE': 'GR_cube', 'VSH_CUBE':'VSH_cube'}).dropna(subset=['GR_cube'])
df_gr_cube = df_gr_cube.round({'GR_cube':2, 'VSH_cube':2})
df_gr_cube.loc[df_gr_cube.VSH_cube < 0, 'VSH_cube'] = 0
df_gr_cube.loc[df_gr_cube.VSH_cube > 1, 'VSH_cube'] = 1
df_gr_cube

# md_interpolation

In [ ]:
def interpolate_md_gr_cube(one_well, step):
    one_well = one_well.sort_values(by='MD')
    well_name = one_well["well"].iloc[0]
    data_range = np.floor((one_well["MD"].max() - one_well["MD"].min())/step)
    starting_md = one_well["MD"].iloc[0]
    new_MD_values = [starting_md + i*0.1 for i in range(1,int(data_range))]

    # interp_MD = interp1d(one_well['MD'], one_well['MD'], kind='linear', fill_value="extrapolate")
    interp_gr_cube = interp1d(one_well['MD'], one_well['GR_cube'], kind='linear', fill_value="extrapolate")
    new_data = {
        'well': [well_name for _ in range(len(new_MD_values))],
        'MD': new_MD_values,
        'GR_cube': interp_gr_cube(new_MD_values)
        }
    new_df = pd.DataFrame(new_data)
    return new_df

df_lst = []
print('Start interpolation')
for wellnames in tqdm(df_gr_cube.well.unique()):
    well_sel = df_gr_cube[df_gr_cube.well == wellnames]
    well_interp = interpolate_md_gr_cube(well_sel, 0.1)
    df_lst.append(well_interp)
df_interp = pd.concat(df_lst)
df_interp = df_interp.round({'MD':1})
df_interp

In [22]:
df_interp.to_csv('C:\jupyter\SPP\inputoutput\gr_cube_md_interp.csv')

In [ ]:
for wellnames in df_gr_cube.well.unique()[:10]:
    fig, ax = plt.subplots(figsize=(3, 10))
    orig = df_gr_cube[(df_gr_cube.well == wellnames) & (df_gr_cube.MD > 2500)]
    interp = df_interp[(df_interp.well == wellnames) & (df_interp.MD > 2500)]
    ax.plot(orig['GR_cube'], orig['MD'], label='Original')
    ax.plot(interp['GR_cube'], interp['MD'], label='Interpolated')
    ax.invert_yaxis()
    ax.legend(title=wellnames)
    plt.show()